# using Meta's SAM

In [ ]:
!pip install git+https://github.com/facebookresearch/segment-anything.git

In [ ]:
!pip install opencv-python pycocotools matplotlib onnxruntime onnx

In [ ]:
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth# https://github.com/facebookresearch/segment-anything#model-checkpoints

In [ ]:
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

In [ ]:
#/kaggle/working/sam_vit_b_01ec64.pth
from segment_anything import SamPredictor, sam_model_registry #import SAM libraries


sam = sam_model_registry["vit_b"](checkpoint="/kaggle/working/sam_vit_b_01ec64.pth") #load model
predictor = SamPredictor(sam) #load predictor

In [ ]:
import matplotlib.pyplot as plt 
img_path = '/kaggle/input/sampledsailporini/sample-dsail-porini/dataset/2021-07-09-09-09-05.jpg'
#img = plt.imread("/kaggle/input/sampledsailporini/dataset/2021-07-09-09-09-05.jpg")
img = plt.imread(img_path)
plt.imshow(img)

In [ ]:
predictor.set_image(img) #set image

In [ ]:
#masks, _, _ = predictor.predict()

In [ ]:
#plotting function
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))    

In [ ]:
import numpy as np
# MANUAL #
input_point =  np.array([[400, 320],[1000,320]]) #375 #what are these points?
input_label = np.array([1,1])
# MANUAL #
plt.figure(figsize=(10,10))
plt.imshow(img)
show_points(input_point, input_label, plt.gca())
plt.axis('on')
plt.show()  

In [ ]:
#predict with SAMPredictor.predict
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=False,
)

In [ ]:
#help(predictor.predict) #XYXY format

In [ ]:
masks.shape  # (number_of_masks) x H x W

In [ ]:
type(masks)

In [ ]:
#help(masks)
mask_of_mask = masks == True
masks[mask_of_mask] = 255.0
mask_of_mask = masks == False
masks[mask_of_mask] = 0.0

In [ ]:
#masks.reshape((masks.shape[2],masks.shape[1],masks.shape[0]))
#masks.shape
#help(np.array)
plt_mask = masks.squeeze()

# Visualize Masks

In [ ]:
#plt.figure(figsize=(10,10))
plt.imshow(plt_mask)
#show_mask(masks, plt.gca())#show_points(input_point, input_label, plt.gca())
#plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
#plt.axis('off')
#plt.show()  

In [ ]:
print(np.unique(plt_mask))

In [ ]:
True-True

In [ ]:
True-False

In [ ]:
img[:,:,0].shape

In [ ]:
plt.imshow(plt_mask*img[:,:,0])

In [ ]:
plt.imshow(plt_mask*img[:,:,1])

In [ ]:
plt.imshow(img[:,:,2]*plt_mask)

In [ ]:
plt_mask

In [ ]:
#np.ndarray(shape=(2,2)

In [ ]:
img

In [ ]:
image_with_background_removes = np.ndarray(shape=img.shape)
image_with_background_removes[:,:,0] = plt_mask*img[:,:,0]
image_with_background_removes[:,:,1] = plt_mask*img[:,:,1]
image_with_background_removes[:,:,2] = plt_mask*img[:,:,2]
plt.imshow(image_with_background_removes)

In [ ]:
image_with_background_removes

# the type of the array may be an issue , yes it was, I made sure that the array is of data type uint8 to support 8-bit colour :D and it worked as seen below

In [ ]:
image_with_background_removes = np.ndarray(shape=img.shape,dtype='uint8')
image_with_background_removes[:,:,0] = plt_mask*img[:,:,0]
image_with_background_removes[:,:,1] = plt_mask*img[:,:,1]
image_with_background_removes[:,:,2] = plt_mask*img[:,:,2]
plt.imshow(image_with_background_removes)

# Add Background to image with no background

In [ ]:
background_mask = 1-plt_mask #get background
#print(plt_mask
plt.imshow(background_mask)

In [ ]:
background_img = plt.imread("/kaggle/input/dsail-porini/DSAIL-Porini Annotated camera trap images of wildlife species from a conservancy in Kenya/RaspberryPi_images/1st-deployment/2021-06-25-14-56-09.jpg")
plt.imshow(background_img[:,:,:])

In [ ]:
image_with_background_removes1 = np.ndarray(shape=img.shape,dtype='uint8')
image_with_background_removes1[:,:,0] = image_with_background_removes[:,:,0]*background_img[:,:,0]
image_with_background_removes1[:,:,1] = image_with_background_removes[:,:,1]*background_img[:,:,1]
image_with_background_removes1[:,:,2] = image_with_background_removes[:,:,2]*background_img[:,:,2]
plt.imshow(image_with_background_removes1)

In [ ]:
image_with_background_removes1 = np.ndarray(shape=img.shape,dtype='uint8')
image_with_background_removes1[:,:,0] = image_with_background_removes[:,:,0]*background_mask
image_with_background_removes1[:,:,1] = image_with_background_removes[:,:,1]*background_mask
image_with_background_removes1[:,:,2] = image_with_background_removes[:,:,2]*background_mask
plt.imshow(image_with_background_removes1)

# Why?

Because I'm using multiplication only use addition.

In [ ]:
image_with_background_removes1 = np.ndarray(shape=img.shape,dtype='uint8')
image_with_background_removes1[:,:,0] = image_with_background_removes[:,:,0]+background_mask*background_img[:,:,0]
image_with_background_removes1[:,:,1] = image_with_background_removes[:,:,1]+background_mask*background_img[:,:,1]
image_with_background_removes1[:,:,2] = image_with_background_removes[:,:,2]+background_mask*background_img[:,:,2]
plt.imshow(image_with_background_removes1)

In [ ]:
image_with_background_removes1

# More Experiments towards automation

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img)
show_mask(plt_mask, plt.gca())
show_points(input_point, input_label, plt.gca())
plt.title(f"Mask, Score:", fontsize=18)
plt.axis('off')
plt.show()  

In [ ]:
for i, (mask, score) in enumerate(zip(masks, scores)):
    plt.figure(figsize=(10,10))
    plt.imshow(img)
    show_mask(mask, plt.gca())
    show_points(input_point, input_label, plt.gca())
    plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
    plt.axis('off')
    plt.show()  

### Single image segemetation

In [ ]:
input_point = np.array([[400, 320],[1000,320]]) #375 #what are these points?
input_label = np.array([1,1])

plt.figure(figsize=(10,10))
plt.imshow(img)
show_points(input_point, input_label, plt.gca())
plt.axis('on')
plt.show()  

In [ ]:
#predict with SAMPredictor.predict
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=False,
)

In [ ]:
masks.shape  # (number_of_masks) x H x W

In [ ]:
masks

In [ ]:
for i, (mask, score) in enumerate(zip(masks, scores)):
    plt.figure(figsize=(10,10))
    plt.imshow(img)
    show_mask(mask, plt.gca())
    show_points(input_point, input_label, plt.gca())
    plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
    plt.axis('off')
    plt.show() 

In [ ]:
input_point = np.array([[500, 100], [1125, 625]])
input_label = np.array([1, 0])

mask_input = logits[np.argmax(scores), :, :]  # Choose the model's best mask
plt.figure(figsize=(10,10))
plt.imshow(img)
show_points(input_point, input_label, plt.gca())
plt.axis('on')
plt.show()  

In [ ]:
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    mask_input=mask_input[None, :, :],
    multimask_output=True,
)

In [ ]:
masks.shape

In [ ]:
mask

In [ ]:
# plt.figure(figsize=(10,10))
# plt.imshow(img)
# show_mask(masks, plt.gca())
# show_points(input_point, input_label, plt.gca())
# plt.axis('off')
# plt.show() 
for i, (mask, score) in enumerate(zip(masks, scores)):
    plt.figure(figsize=(10,10))
    plt.imshow(img)
    show_mask(mask, plt.gca())
    show_points(input_point, input_label, plt.gca())
    plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
    plt.axis('off')
    plt.show()  

# Batched prompt inputs

SamPredictor can take multiple input prompts for the same image, using predict_torch method. This method assumes input points are already torch tensors and have already been transformed to the input frame. For example, imagine we have several box outputs from an object detector.

In [ ]:
import torch
import cv2

In [ ]:
input_boxes = torch.tensor([
    [75, 275, 1725, 850],
    [425, 600, 700, 875],
    [1375, 550, 1650, 800],
    [1240, 675, 1400, 750],
], device=predictor.device)

Transform the boxes to the input frame, then predict masks. SamPredictor stores the necessary transform as the transform field for easy access, though it can also be instantiated directly for use in e.g. a dataloader (see segment_anything.utils.transforms).

In [ ]:
transformed_boxes = predictor.transform.apply_boxes_torch(input_boxes, img.shape[:2])
masks, _, _ = predictor.predict_torch(
    point_coords=None,
    point_labels=None,
    boxes=transformed_boxes,
    multimask_output=False,
)

In [ ]:
masks.shape  # (batch_size) x (num_predicted_masks_per_input) x H x W

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(img)
for mask in masks:
    show_mask(mask.cpu().numpy(), plt.gca(), random_color=True)
for box in input_boxes:
    show_box(box.cpu().numpy(), plt.gca())
plt.axis('off')
plt.show()

In [ ]:
#help(predictor)


# YOLOv8 Object detection

In [ ]:
!pip install ultralytics
!pip install -U ipywidgets

In [ ]:
import random
import os
path = "/kaggle/input/sampledsailporini/sample-dsail-porini/dataset/"
#image_path = path + random.choice(os.listdir(path))
image_path = '/kaggle/input/sampledsailporini/sample-dsail-porini/dataset/2021-07-09-09-09-05.jpg'

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# Load a model
model = YOLO('yolov8n.pt')  # load an official model
img = plt.imread(image_path)
# Predict with the model
results = model(image_path)  # predict on an image

In [ ]:
plt.imshow(results[0].plot())
plt.axis('off')
# Add title
title="Object Detection Example"
plt.title(title)
# Show the plot
plt.show()

In [ ]:
#help(results[0].plot)

In [ ]:
plt.imshow(results[0].plot(labels=False))
plt.axis('off')
# Add title
title="Object Detection Example"
plt.title(title)
# Show the plot
plt.show()

# Get yolo format bounding boxes

In [ ]:
results[0].boxes.xywh

# Obtain XYXY bounding boxes

In [ ]:
xyxy = results[0].boxes.xyxy
print(xyxy)
xyxy = [np.array(i) for i in xyxy]
print(xyxy)

In [ ]:
def generate_points_from_boxes(coords_results):
    centers = []
    for coords in coords_results.xyxy:
        x1 = coords[0]
        y1 = coords[1]
        x2 = coords[2]
        y2 = coords[3]

        centerx,centery = ((x1 + x2) / 2, (y1 + y2) / 2 ) #( np.average(coords[:2]),np.average(coords[2:]))
        centers.append([centerx,centery])
    return centers

In [ ]:
centers = generate_points_from_boxes(results[0].boxes)

# Show Points Generated from Boxes

In [ ]:
import numpy as np
# MANUAL #
input_point =  np.array(centers) #375 #what are these points?
input_label = np.array([1]*len(centers))
# MANUAL #
plt.figure(figsize=(10,10))
plt.imshow(img)
show_points(input_point, input_label, plt.gca())
plt.axis('on')
plt.show()  

In [ ]:
#predict with SAMPredictor.predict
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=False,
)

In [ ]:
plt_mask = masks.squeeze()
plt.imshow(plt_mask)

In [ ]:
def remove_background(img, plt_mask):
    image_with_background_removes = np.ndarray(shape=img.shape,dtype='uint8')
    image_with_background_removes[:,:,0] = plt_mask*img[:,:,0]
    image_with_background_removes[:,:,1] = plt_mask*img[:,:,1]
    image_with_background_removes[:,:,2] = plt_mask*img[:,:,2]
    return image_with_background_removes
image_with_background_removed = remove_background(img, plt_mask)
plt.imshow(image_with_background_removed)

In [ ]:
# plt.imshow(results[0].plot())
# plt.axis('off')
# # Add title
# title="Object Detection Example"
# plt.title(title)
# # Show the plot
# plt.show()

# YOLOv8 segmentation (Not necessary at the moment)

In [ ]:
!pip install ultralytics
!pip install -U ipywidgets

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n-seg.pt')  # load an official model
image_path="/kaggle/input/sampledsailporini/sample-dsail-porini/dataset/2021-07-09-09-09-05.jpg"
# Predict with the model
results = model(image_path)  # predict on an image

In [ ]:
plt.imshow(results[0].plot())
plt.axis('off')
# Add title
title="Segmentation Example"
plt.title(title)
# Show the plot
plt.show()

# segmentation masks only

In [ ]:
plt.imshow(results[0].plot(boxes=False))
plt.axis('off')
# Add title
title="Segmentation Example"
plt.title(title)
# Show the plot
plt.show()

In [ ]:
#help(results[0].masks)

In [ ]:
#help(results[0])

# Using Megadetector

In [ ]:
# !sudo apt-get update -y
# !sudo apt-get install python3.8 python3.8-dev python3.8-distutils libpython3.8-dev -y

# #change alternatives
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1 -y
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2 -y

# #Check that it points at the right location
# !python3 --version

In [ ]:
# # install pip
# !curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
# !python3 get-pip.py --force-reinstall

# #install colab's dependencies
# !python3 -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

In [ ]:
# import torch
# #!pip install torch==1.10.1
# !pip install PytorchWildlife
# from PytorchWildlife.models import detection as pw_detection
# from PytorchWildlife.models import classification as pw_classification

# img = torch.randn((3, 1280, 1280))

# # Detection
# detection_model = pw_detection.MegaDetectorV5() # Model weights are automatically downloaded.
# detection_result = detection_model.single_image_detection(img)